# Training models
On the previously constructed dataset, we will train the following classificaiton models:
- VGG16
- MobileNet
- ResNet50

Further we will define our own model and train it.


In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNet, ResNet50  # Using ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

Num GPUs Available:  1
Num CPUs Available:  1


In [3]:
# Parameters
#data_dir = "../dat/images_augmented_cropped" 
data_dir = "../dat/gempundit_2022_cropped_augmented_2000"
target_size = (224, 224)
batch_size = 32
num_epochs = 10

validation_split = 0.2  # Percentage of data to use for validation

# Data Generator with Splitting
datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)

# Load data with the split
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' 
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' 
)

Found 164657 images belonging to 99 classes.
Found 41150 images belonging to 99 classes.


In [ ]:
# Model Creation with Pre-trained Weights
def create_model(model_type):
    if model_type == "VGG16":
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=target_size + (3,))
    elif model_type == "MobileNet":
        base_model = MobileNet(include_top=False, weights='imagenet', input_shape=target_size + (3,))
    elif model_type == "ResNet50":
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=target_size + (3,))
    elif model_type == "OwnModel":
        kernel_size =  3  # Example value
        max_pool =  2  # Example value
        base_model = Sequential([
            # First layer
            Conv2D(32, (kernel_size, kernel_size), activation='relu', padding='same', input_shape=target_size + (3,)),
            MaxPooling2D((max_pool, max_pool)),
            # Second layer
            Conv2D(64, (kernel_size, kernel_size), activation='relu', padding='same'),
            MaxPooling2D((max_pool, max_pool)),
            # Third layer
            Conv2D(128, (kernel_size, kernel_size), activation='relu', padding='same'),
            MaxPooling2D((max_pool, max_pool)),
            # Fourth layer
            Conv2D(128, (kernel_size, kernel_size), activation='relu', padding='same'),
            AveragePooling2D(pool_size=(2,  2), strides=(2,  2)),
            # Fifth layer
            Conv2D(128, (kernel_size, kernel_size), activation='relu', padding='same'),
            MaxPooling2D((max_pool, max_pool)),
])

    else:
        raise ValueError("Invalid model_type")

    model = keras.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(train_data.num_classes, activation='softmax') 
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_accuracy", dtype=None), tf.keras.metrics.TopKCategoricalAccuracy(k=5, name="top_5_accuracy", dtype=None)])
    return model

# Training and Evaluation
models = {
    "VGG16": create_model("VGG16"),
    "MobileNet": create_model("MobileNet"),
    "ResNet50": create_model("ResNet50"),
    "OwnModel": create_model("OwnModel"),
}

results = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    history = model.fit(train_data, epochs=num_epochs, validation_data=val_data)
    results[model_name] = history.history

    # Save model parameters
    model.save(f"../mod/{model_name}.h5")
    # Convert history to DataFrame
    df = pd.DataFrame(history)
    
    # Save DataFrame to CSV
    df.to_csv(f"../mod/{model_name}_history.csv", index=False)

print("Training and Evaluation Complete!")


## Finetune pre-trained models

In [25]:
model_dir = r'..\mod\first_run_gempundit_2022_cropped_augmented_2000'
models = ['VGG16', 'MobileNet', 'ResNet50']
results = {}
for model_name in models:
    model = keras.models.load_model(os.path.join(model_dir, f"{model_name}.h5"))
    model.trainable = True

    print(f"Training {model_name}...")
    

    model.compile(keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', 
                  metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3_accuracy", dtype=None), tf.keras.metrics.TopKCategoricalAccuracy(k=5, name="top_5_accuracy", dtype=None)])

    history = model.fit(train_data, epochs=5, validation_data=val_data)
    results[model_name] = history.history

    # Save model parameters
    model.save(os.path.join(model_dir, f"{model_name}_FT_llr.h5"))
    # Convert history to DataFrame
    df = pd.DataFrame(history.history)

    # Save DataFrame to CSV55
    df.to_csv(os.path.join(model_dir, f"{model_name}_FT_llr.csv"))

Training VGG16...
Epoch 1/5
5146/5146 [==============================] - 1472s 286ms/step - loss: 0.5451 - accuracy: 0.7460 - top_3_accuracy: 0.9846 - top_5_accuracy: 0.9969 - val_loss: 2.7432 - val_accuracy: 0.4914 - val_top_3_accuracy: 0.8165 - val_top_5_accuracy: 0.8855
Epoch 2/5
5146/5146 [==============================] - 1726s 335ms/step - loss: 0.4953 - accuracy: 0.7583 - top_3_accuracy: 0.9898 - top_5_accuracy: 0.9984 - val_loss: 2.5033 - val_accuracy: 0.5165 - val_top_3_accuracy: 0.8452 - val_top_5_accuracy: 0.9068
Epoch 3/5
5146/5146 [==============================] - 1633s 317ms/step - loss: 0.4715 - accuracy: 0.7656 - top_3_accuracy: 0.9923 - top_5_accuracy: 0.9990 - val_loss: 2.6623 - val_accuracy: 0.5110 - val_top_3_accuracy: 0.8376 - val_top_5_accuracy: 0.8995
Epoch 4/5
5146/5146 [==============================] - 1826s 355ms/step - loss: 0.4536 - accuracy: 0.7707 - top_3_accuracy: 0.9933 - top_5_accuracy: 0.9992 - val_loss: 2.8001 - val_accuracy: 0.5062 - val_top_3_accu